In [ ]:
import os
from glob import glob
root_dir = r'C:\Users\jnicolow\Documents\research\CRC\sat_img_seg_annotation\data\sat_images'

In [ ]:
import os
import rasterio
from shapely.geometry import Point
import geojson
import pyproj

def get_centroid_of_raster(tiff_path):
    try:
        with rasterio.open(tiff_path) as src:
            # Get the metadata (e.g., width, height, transform)
            width = src.width
            height = src.height
            transform = src.transform

            # Calculate the centroid (middle of the raster)
            x_center = transform[2] + transform[0] * width / 2
            y_center = transform[5] + transform[4] * height / 2

            # Convert (x, y) to (lat, lon)
            # pyproj used for transforming from raster CRS to WGS84
            projection = pyproj.CRS(src.crs)  # Raster's CRS
            wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 lat/lon

            transformer = pyproj.Transformer.from_crs(projection, wgs84, always_xy=True)
            lon, lat = transformer.transform(x_center, y_center)

            return lon, lat
    except Exception as e:
        print(f"Error opening {tiff_path}: {e}")
        return None

def create_geojson_from_centroids(base_dir):
    features = []

    # Walk through all subdirectories and files
    for subdir, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.tif') or file.endswith('.tiff'):
                tiff_path = os.path.join(subdir, file)
                
                # Get the centroid for the current TIFF
                centroid = get_centroid_of_raster(tiff_path)
                
                if centroid:
                    # Create a Point feature from the centroid (lat, lon)
                    point = Point(centroid)
                    feature = {
                        'type': 'Feature',
                        'geometry': point.__geo_interface__,
                        'properties': {
                            'filename': file
                        }
                    }
                    features.append(feature)

    # Create the GeoJSON structure
    geojson_data = {
        'type': 'FeatureCollection',
        'features': features
    }

    # Write GeoJSON to a file
    with open('centroids.geojson', 'w') as f:
        geojson.dump(geojson_data, f)

    print("GeoJSON file 'centroids.geojson' created.")

# Example usage
base_dir = 'path_to_your_folder_with_tiffs'  # Replace with your actual folder path
create_geojson_from_centroids(base_dir)


In [16]:
import os
import rasterio
import geojson
import pyproj
from shapely.geometry import Point
import json

def get_centroid_of_raster(tiff_path):
    try:
        with rasterio.open(tiff_path) as src:
            # Get the metadata (e.g., width, height, transform)
            width = src.width
            height = src.height
            transform = src.transform

            # Calculate the centroid (middle of the raster)
            x_center = transform[2] + transform[0] * width / 2
            y_center = transform[5] + transform[4] * height / 2

            # Convert (x, y) to (lat, lon)
            # pyproj used for transforming from raster CRS to WGS84
            projection = pyproj.CRS(src.crs)  # Raster's CRS
            wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 lat/lon

            transformer = pyproj.Transformer.from_crs(projection, wgs84, always_xy=True)
            lon, lat = transformer.transform(x_center, y_center)

            return lon, lat
    except Exception as e:
        print(f"Error opening {tiff_path}: {e}")
        return None

def extract_first_polygon_coordinates(geojson_path):
    coordinates = []
    try:
        with open(geojson_path, 'r') as f:
            data = json.load(f)
            # Extract the first coordinate of each polygon in the GeoJSON
            for feature in data['features']:
                if feature['geometry']['type'] == 'Polygon':
                    first_coord = feature['geometry']['coordinates'][0][0]  # First coordinate of first polygon
                    coordinates.append(first_coord)
        return coordinates
    except Exception as e:
        print(f"Error reading {geojson_path}: {e}")
        return []

def create_geojson_from_centroids_and_polygon(base_dir, siteinfo_dir):
    features = []

    # Walk through all GeoJSON files in the siteinfo directory
    for subdir, _, files in os.walk(siteinfo_dir):
        for file in files:
            if file.endswith('.geojson'):
                geojson_path = os.path.join(subdir, file)
                
                # Extract the first coordinates from the GeoJSON
                polygon_coordinates = extract_first_polygon_coordinates(geojson_path)
                
                for coord in polygon_coordinates:
                    # Add the polygon's first coordinate as a Point feature
                    point = Point(coord)
                    feature = {
                        'type': 'Feature',
                        'geometry': point.__geo_interface__,
                        'properties': {
                            'name': 'Polygon first coordinate'
                        }
                    }
                    features.append(feature)

    # Walk through all subdirectories and files for TIFFs (from base_dir)
    for subdir, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.tif') or file.endswith('.tiff'):
                tiff_path = os.path.join(subdir, file)
                
                # Get the centroid for the current TIFF
                centroid = get_centroid_of_raster(tiff_path)
                
                if centroid:
                    # Create a Point feature from the centroid (lat, lon)
                    point = Point(centroid)
                    feature = {
                        'type': 'Feature',
                        'geometry': point.__geo_interface__,
                        'properties': {
                            'filename': file
                        }
                    }
                    features.append(feature)

    # Create the GeoJSON structure
    geojson_data = {
        'type': 'FeatureCollection',
        'features': features
    }

    # Write GeoJSON to a file
    with open('centroids_and_polygons.geojson', 'w') as f:
        geojson.dump(geojson_data, f)

    print("GeoJSON file 'centroids_and_polygons.geojson' created.")

# Example usage
base_dir = root_dir  # Replace with your actual folder path
siteinfo_dir = 'data/siteinfo'  # Path to the siteinfo folder containing multiple geojsons
create_geojson_from_centroids_and_polygon(base_dir, siteinfo_dir)


Error opening C:\Users\jnicolow\Documents\research\CRC\sat_img_seg_annotation\data\sat_images\waikikihazardous2020\20200720_202108_53_2257_3B_AnalyticMS_toar_clip.tif: 'C:/Users/jnicolow/Documents/research/CRC/sat_img_seg_annotation/data/sat_images/waikikihazardous2020/20200720_202108_53_2257_3B_AnalyticMS_toar_clip.tif' not recognized as a supported file format.
GeoJSON file 'centroids_and_polygons.geojson' created.
